In [ ]:
import pandas as pd
import xdem
import glob
import matplotlib.pyplot as plt
import geoutils as gu
from scipy import stats
from scipy.stats import skew, kurtosis
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors  # Import LogNorm from matplotlib
from SebVel.utils import calculate_bin_averages, create_polar_plot_with_bins

In [ ]:
filtered= True
its= True
DEM_Alps = '/home/charriel/Documents/PostdocIGE/Seasonality/metadata/DEM/Copernicus30m/DEM_30m_cropped_for_illumination.tif'  #DEM used to compute slope and aspect

In [ ]:
if not filtered: path_tiff = '/home/charriel/Documents/PostdocIGE/Seasonality/polar_plot/vy_vy_sinus_notfitered_IGE/'
else: path_tiff = '/home/charriel/Documents/PostdocIGE/Seasonality/polar_plot/vx_vy_fit_sinus_filtered_IGE'
if its:  path_tiff = '/home/charriel/Documents/PostdocIGE/Seasonality/polar_plot/vx_vy_fit_sinus_ITS_LIVE_S2'

In [ ]:
#TO READ THE GEOTIFF AMPLITUDES
amplitudex = f'{path_tiff}/amplitudex_cropped_massif.tif'
amplitudey = f'{path_tiff}/amplitudey_cropped_massif.tif'
amplitudev = f'{path_tiff}/amplitudev_cropped_massif.tif'

#load
amplitudex, dem_alps, amplitudev, amplitudey = gu.raster.load_multiple_rasters(
    [amplitudex, DEM_Alps, amplitudev, amplitudey], crop=True,
    ref_grid=0)

In [ ]:

## POLAR PLOTS, on 4 quadrants

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors  # Import LogNorm from matplotlib

# Example data, replace these with your actual slope gradient, aspect, and amplitude data
aspect_angles = aspect.data[aspect.data.mask == False]
slope_angles = slope.data[aspect.data.mask == False]
EW_amplitude = amplitudex.data[aspect.data.mask == False]  # Replace with your actual EW amplitude data
NS_amplitude = amplitudey.data[aspect.data.mask == False]  # Replace with your actual NS amplitude data
vv_amplitude = amplitudev.data[aspect.data.mask == False]
# vmax = 30
vmax = 100
vmin = 0.1  # Avoid zero for logarithmic scaling
criterion = '95'


# Function to calculate average amplitude per 10x10 degree bin
def calculate_bin_averages(aspect_angles, slope_angles, amplitudes, criterion='mean'):
    bins_aspect = np.arange(0, 370, 90)
    bins_slope = np.arange(0, 120, 30)
    bin_averages = np.zeros((len(bins_slope) - 1, len(bins_aspect) - 1))
    bin_number_of_values = np.zeros((len(bins_slope) - 1, len(bins_aspect) - 1))

    for i in range(len(bins_aspect) - 1):
        for j in range(len(bins_slope) - 1):
            mask = ((aspect_angles >= bins_aspect[i]) & (aspect_angles < bins_aspect[i + 1]) &
                    (slope_angles >= bins_slope[j]) & (slope_angles < bins_slope[j + 1]))
            bin_values = amplitudes[mask]
            bin_number_of_value = len(bin_values)
            if len(bin_values) > 0:
                if criterion == 'mean':
                    bin_averages[j, i] = np.nanmean(bin_values)
                elif criterion == 'median':
                    bin_averages[j, i] = np.nanmedian(bin_values)
                elif criterion == 'std':
                    bin_averages[j, i] = np.std(bin_values)
                elif criterion == 'MAD':
                    bin_averages[j, i] = stats.median_abs_deviation(bin_values)
                elif criterion == '90':
                    bin_averages[j, i] = np.percentile(bin_values, 90)
                elif criterion == '95':
                    bin_averages[j, i] = np.percentile(bin_values, 95)
                bin_number_of_values[j, i] = bin_number_of_value
            else:
                bin_averages[j, i] = np.nan  # Handle empty bins
                bin_number_of_values[j, i] = np.nan

    return bins_aspect, bins_slope, bin_averages, bin_number_of_values


# Function to create polar plot with bin averages
def create_polar_plot_with_bins(bin_averages, bins_aspect, bins_slope, title, color_map, vmax, subplot_position,
                                norm='normal', one_plot=False):
    if not one_plot:
        ax = plt.subplot(1, 2, subplot_position, projection='polar')
    else:
        ax = plt.subplot(projection='polar')
    theta, r = np.meshgrid(np.deg2rad(bins_aspect), 90 - bins_slope)

    # Apply logarithmic scale using LogNorm
    if norm == 'log':
        norm = colors.LogNorm(vmin=1, vmax=vmax)
        c = ax.pcolormesh(theta, r, bin_averages, cmap=color_map, norm=norm, shading='auto')
    else:
        c = ax.pcolormesh(theta, r, bin_averages, cmap=color_map, vmin=0, vmax=vmax, shading='auto')
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.set_title(title, fontsize=16, pad=50)
    plt.colorbar(c, ax=ax, pad=0.1, orientation='vertical').set_label('Seasonal Amplitude (m/y)', fontsize=16)
    r_labels = np.arange(0, 90, 30)  # Adjust step size as needed
    ax.set_yticks(80 - r_labels)  # Position ticks accordingly
    ax.set_yticklabels(r_labels, fontsize=16)  # Label them correctly
    ax.set_xticks(np.deg2rad([0, 90, 180, 270]))  # Set the custom angle ticks
    ax.set_xticklabels(['N', 'E', 'S', 'W'], fontsize=16)  # Label them as N, E, S, W

    # Position the y-tick labels more prominently at the top
    for label in ax.get_yticklabels():
        label.set_verticalalignment('bottom')


# Calculate bin averages
bins_aspect, bins_slope, EW_bin_averages, EW_bin_number_of_values = calculate_bin_averages(aspect_angles, slope_angles,
                                                                                           EW_amplitude,
                                                                                           criterion=criterion)
_, _, NS_bin_averages, NS_bin_number_of_values = calculate_bin_averages(aspect_angles, slope_angles, NS_amplitude,
                                                                        criterion=criterion)
_, _, vv_bin_averages, vv_bin_number_of_values = calculate_bin_averages(aspect_angles, slope_angles, vv_amplitude,
                                                                        criterion=criterion)
# Plot configuration
fig = plt.figure(figsize=(15, 7))
# Plot EW Seasonal Amplitude with bin averages
create_polar_plot_with_bins(EW_bin_averages, bins_aspect, bins_slope, 'EW', 'inferno', vmax, 1)

# Plot NS Seasonal Amplitude with bin averages
create_polar_plot_with_bins(NS_bin_averages, bins_aspect, bins_slope, 'NS', 'inferno', vmax, 2)
plt.tight_layout()
fig.savefig(f"{path_tiff}/Polar_plot_amplitude_{criterion}.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

fig = plt.figure(figsize=(15, 7))
create_polar_plot_with_bins(vv_bin_averages, bins_aspect, bins_slope, '', 'inferno', vmax=vmax, subplot_position=2,
                            one_plot=True)
plt.tight_layout()
fig.savefig(f"{path_tiff}/Polar_plot_amplitude_{criterion}_vv_shorter.png", dpi=300, bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(15, 7))
# Plot EW Seasonal Amplitude with bin number of values
create_polar_plot_with_bins(EW_bin_number_of_values, bins_aspect, bins_slope, 'EW Seasonal Amplitude - Number of data',
                            'inferno', 5000, 1, norm='log')

# Plot NS Seasonal Amplitude with bin number of values
create_polar_plot_with_bins(NS_bin_number_of_values, bins_aspect, bins_slope, 'NS Seasonal Amplitude - Number of data',
                            'inferno', 5000, 2, norm='log')
plt.tight_layout()
fig.savefig(f"{path_tiff}/Polar_plot_number_of_values_shorter.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
path_save_file = '/home/charriel/Documents/PostdocIGE/Seasonality/shadows_projection/interpolation_of_stable_areas_results_to_glaciers/'

if filtered:name_save_file = f'bins_averages_{criterion}_ige_filtered'
else: name_save_file = f'bins_averages_{criterion}_ige_not_filtered'
if its: name_save_file = f'bins_averages_{criterion}_its_live_s2'


EW_bin_averages_df = pd.DataFrame(EW_bin_averages)
# Assigning index and column values
EW_bin_averages_df.index = np.arange(30, 120, 30)  # Set index values
EW_bin_averages_df.columns = np.arange(90, 370, 90)  # Set column names

# Assigning names to index and columns
EW_bin_averages_df.index.name = "Slope Bins"  # Name for the index
EW_bin_averages_df.columns.name = "Aspect Bins"  # Name for the columns
EW_bin_averages_df.to_csv(f'{path_save_file}/EW_{name_save_file}.csv')

NS_bin_averages_df = pd.DataFrame(NS_bin_averages)
# Assigning index and column values
NS_bin_averages_df.index = np.arange(30, 120, 30)  # Set index values
NS_bin_averages_df.columns = np.arange(90, 370, 90)  # Set column names
# Assigning names to index and columns
NS_bin_averages_df.index.name = "Slope Bins"  # Name for the index
NS_bin_averages_df.columns.name = "Aspect Bins"  # Name for the columns
NS_bin_averages_df.to_csv(
    f'{path_save_file}/NS_{name_save_file}.csv')

NS_bin_averages_df = pd.DataFrame(vv_bin_averages)
# Assigning index and column values
NS_bin_averages_df.index = np.arange(30, 120, 30)  # Set index values
NS_bin_averages_df.columns = np.arange(90, 370, 90)  # Set column names
# Assigning names to index and columns
NS_bin_averages_df.index.name = "Slope Bins"  # Name for the index
NS_bin_averages_df.columns.name = "Aspect Bins"  # Name for the columns
NS_bin_averages_df.to_csv(
    f'{path_save_file}/vv_{name_save_file}.csv')